# Example of use

In [1]:
import Skydipper

In [2]:
Skydipper.__version__

'0.1.1'

## Dataset objects

Using known id's you can instantiate a dataset object directly.

In [5]:
ds = Skydipper.Dataset(id_hash='94241cb4-9e91-4a1a-9fcc-993b8ac9c2b7')
ds

Dataset 94241cb4-9e91-4a1a-9fcc-993b8ac9c2b7 bio.014 Biodiversity Intactness

You can access the attributes of a dataset.

In [ ]:
ds.attributes

You can also access the metadata and vocabularies if they exist.

In [ ]:
ds.vocabulary[0].attributes

In [ ]:
ds.metadata[0].attributes

## Queries on Datasets

Datasets can be queried via SQL, with a table returned. Currently this is only supported for Carto-type data:

In [ ]:
d = Skydipper.Dataset(id_hash='bd5d7924-611e-4302-9185-8054acb0b44b')
d

In [ ]:
d.query('SELECT fid, ST_ASGEOJSON(the_geom_webmercator) FROM data LIMIT 5')

## Layer Objects

Similarly, you can also instantiate a Layer object.

In [ ]:
ly = Skydipper.Layer(id_hash='dc6f6dd2-0718-4e41-81d2-109866bb9edd')
ly

Layers can be visulized if appropriate via a call to Layer().map()

In [ ]:
ly.map()

## Tables

Tables are subclasses of Dataset objects. They are document datasets which can be instantiated and queried returning a dataframe object.

In [ ]:
t = Skydipper.Table(id_hash='86c7135a-855d-4f1b-9d67-f545a93281b3')
t

In [ ]:
df = t.head(3)
df

Queries to tables are returned in geopandas dataframe format.

In [ ]:
type(df)

In [ ]:
t.query("SELECT * from data where City = 'San Francisco'")

## Collection objects: Searching

If you don't know what data you are interested in advance, you can search by keywords and return a list of objects.

In [ ]:
c = Skydipper.Collection('tree cover', object_type=['layer','dataset'], app=['gfw'], limit=10)

In [ ]:
c

Searching can be restricted with keyword arguments to specifically search types of items, applications, and more. If you want to render those items, you will need to do the following.

You can access items from a collection using subscripts, slices and more. Note that slicing, or selecting by element instantiates the Layer, Table, or Dataset object.

In [ ]:
c[0:3]

In [ ]:
c[-1]

In [ ]:
l = c[-1]
l.map()

## Create a Geometry object

Often you will need to perform some kind of intersect analysis between data held in datasets and tables and a geometry.
We will now show you multiple ways to create your geometry objects.

### From an ID

Vizzuality's API holds geometry objects as a Geostore item. Geostore items are accessed by an id-hash. If you know the hash of your object already you can simply call a geometry like so:

In [ ]:
g = Skydipper.Geometry(id_hash='e8b6f974bcab5aefccd121654860be06')
g

### Geometry attributes

The attributes can be accessed as a dictionary.

In [ ]:
g.attributes

### Geometry as a Table

Table method returns a dataframe of the geometry object. Map will add a Folium map with the geomerty rendered.

In [ ]:
g.table()

### Mapping the Geometry

Calling .map() will create a Folium map with the geomerty rendered.

In [ ]:
g.map()

### From Geojson - Points

You can create an object as you need on the fly from geojson. The act of creating an object will also register it to a Geostore service of your choice (locally, or on a remote server). You can create a geometry object from geojson Points and MultiPoints type data as follows:

In [ ]:
atts = {'geojson': {'type': 'FeatureCollection',
  'features': [{'type': 'Feature',
    'properties': {},
    'geometry': {'type': 'MultiPoint', 'coordinates': [[-4.29, 39.1097]]}}]}}

point = Skydipper.Geometry(attributes=atts, server='https://production-api.globalforestwatch.org')
point

### From Geojson - Polygons

You can create an object as you need on the fly from geojson. The act of creating an object will also register it to a Geostore service of your choice (locally, or on a remote server).
You can create a geometry object from Geojson Polygon and Multipolygon type data as follows:

In [ ]:
atts={'geojson': {'type': 'FeatureCollection',
  'features': [{'type': 'Feature',
    'properties': {},
    'geometry': {'type': 'Polygon',
     'coordinates': [[[82.265625, 32.84267363195431],
       [77.34374999999999, 27.059125784374068],
       [85.4296875, 22.268764039073968],
       [90.3515625, 28.304380682962783],
       [87.5390625, 32.54681317351514],
       [82.265625, 32.84267363195431]]]}}]}}


g1 = Skydipper.Geometry(attributes=atts)
g1

In [ ]:
g1.map()

### From a Shapely object

Shapely objects are at the root of popular python geolibraries such as Geopandas. We can recieve those geometry objects and create a Geometry object (simultaneously registering it in a Vizzuality Geostore server).

In [ ]:
import geopandas as gpd

In [ ]:
%%writefile ./sample.geojson
{"features":[{"properties":null,"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-43.1343734264374,-8.07358087603511],[-43.1327533721924,-8.08277985402466],[-43.1298887729645,-8.08181322762719],[-43.1103515625,-8.07815914647929],[-43.1094932556152,-8.07799981079283],[-43.1094932556152,-8.09641859926744],[-43.1103515625,-8.09645046495416],[-43.1187307834625,-8.0967372560211],[-43.1186878681183,-8.10273857778317],[-43.1186771392822,-8.10358831522616],[-43.1476235389709,-8.10358831522616],[-43.1477630138397,-8.10273857778317],[-43.1505310535431,-8.08645513764317],[-43.1517112255096,-8.08057041885644],[-43.1439757347107,-8.0795931648273],[-43.1448876857758,-8.07574785969913],[-43.1343734264374,-8.07358087603511]]]}}],"crs":{},"type":"FeatureCollection"}

In [ ]:
df = gpd.read_file('./sample.geojson')
df

In [ ]:
s = df.geometry[0]
print(f"Hello! 👋 I am a {type(s)}")
s

In [ ]:
g = Skydipper.Geometry(s=s)

In [ ]:
g

### From Political Boundaries

We are able to return political boundaries (based on [GADM](https://gadm.org/) data) using ISO, admin keys, down to admin-2 level. These should be passed in a dictionary to params. GADM 3.6 is currently used by default.

In [ ]:
params={
    'iso': 'BRA',
    'adm1': None,
    'adm2': None
}
admin = Skydipper.Geometry(parameters=params)
admin.map()

### From an admin geometry with an older GADM version 

Alternatively, you can specify a different gadm version.

In [ ]:

params={
    'iso': 'BRA',
    'adm1': 1,
    'adm2': 1,
    'gadm': '2.7'
}
admin_v1 = Skydipper.Geometry(parameters=params)
admin_v1.map()

### From a Carto table and index

You can also return geometries directly from a Carto table (under the public WRI-01 account) using the table name and cartodb_id.


In [ ]:
params={
    'table': 'gfw_logging',
    'id': 123
}
row_geom = Skydipper.Geometry(parameters=params)
row_geom.map()

## Describe a Geometry 

Return a title and textual description of a geometry in any language.

In [ ]:
g.map()

In [ ]:
%%time
g.describe()

In [ ]:
g.describe(lang='es') # same description but this time in Spanish

Second example of geodescriber, but considering a larger area.

In [ ]:
atts={'geojson': {'type': 'FeatureCollection',
  'features': [{'type': 'Feature',
    'properties': {},
    'geometry': {'type': 'Polygon',
     'coordinates': [[[82.265625, 32.84267363195431],
       [77.34374999999999, 27.059125784374068],
       [85.4296875, 22.268764039073968],
       [90.3515625, 28.304380682962783],
       [87.5390625, 32.54681317351514],
       [82.265625, 32.84267363195431]]]}}]}}


g2 = Skydipper.Geometry(attributes=atts)
g2

In [ ]:
g2.map()

In [ ]:
g2.describe()

After running describe, the results are avaiable as geometry.description

In [ ]:
g2.description

## Add a satellite image to the map of your Geometry

### Single images

The Geometry().map() method supports the return of recent satellite imagery. We calculate the centroid of your geometry and return the best, most recent image that intersects with that point. This is simply to provide context to your geometry. For point geometries we return the best cloud-free image we can find within a specified date range.

In [ ]:
# Create a point

atts = {'geojson': {'type': 'FeatureCollection',
  'features': [{'type': 'Feature',
    'properties': {},
    'geometry': {'type': 'MultiPoint', 'coordinates': [[-4.29, 39.1097]]}}]}}

point = Skydipper.Geometry(attributes=atts, server='https://production-api.globalforestwatch.org')
point

In [ ]:
point.map(image=True, start='2018-01-01', end='2018-12-31')

### Composite images

Whereas for polygon-type geoms we return a cloud-free composite image clipped to your geometry.

In [ ]:
# Create a polygon

atts={'geojson': {'type': 'FeatureCollection',
  'features': [{'type': 'Feature',
    'properties': {},
    'geometry': {'type': 'Polygon',
     'coordinates': [[[-0.87890625, 43.329173667843904],
       [-1.6149902343749998, 42.75104599038353],
       [-1.1865234375, 42.35854391749705],
       [-0.6427001953125, 42.755079545072135],
       [-0.45043945312499994, 42.9524020856897],
       [-0.87890625, 43.329173667843904]]]}}]}}

g1 = Skydipper.Geometry(attributes=atts, server='https://production-api.globalforestwatch.org')
g1

In [ ]:
g1.map(image=True, instrument='sentinel')

## Display a Layer and Geometry together

In [ ]:
atts={'geojson': {'type': 'FeatureCollection',
  'features': [{'type': 'Feature',
    'properties': {},
    'geometry': {'type': 'Polygon',
     'coordinates': [[[-48.131103515625, -0.6001172008725418],
       [-48.13934326171875, -0.9791088369866402],
       [-47.86331176757812, -0.9818550168696459],
       [-47.616119384765625, -0.8459165322899671],
       [-47.50213623046875, -0.7182123915862891],
       [-47.51861572265625, -0.5287095375108173],
       [-47.882537841796875, -0.5163504323777461],
       [-48.131103515625, -0.6001172008725418]]]}}]}}
g = Skydipper.Geometry(attributes=atts)
l = Skydipper.Layer(id_hash='3ef039ba-29c7-459d-aadd-66d6e6cc8f5e')
l

In [ ]:
l.map(geometry=g)

## Intersecting Raster data with Geometry objects

Any Layer or Dataset objects based on an Earth Engine raster can be intersected with Skydipper.Geometry objects.

### Dataset Intersections
In the first instance we show the result of an intersect between a geometry roughly bounding London, and a Population density layer.

In [ ]:
atts={'geojson': {'type': 'FeatureCollection',
  'features': [{'type': 'Feature',
    'properties': {},
    'geometry': {'type': 'Polygon',
     'coordinates': [
          [
            [
              -0.5712890625,
              51.28940590271679
            ],
            [
              0.48339843749999994,
              51.28940590271679
            ],
            [
              0.48339843749999994,
              51.781435604431195
            ],
            [
              -0.5712890625,
              51.781435604431195
            ],
            [
              -0.5712890625,
              51.28940590271679
            ]
          ]
        ]}}]}}

g = Skydipper.Geometry(attributes=atts)


In [ ]:
ds = Skydipper.Dataset(id_hash='a4e9c32d-d037-4c50-a893-967cad193537')
ds

In [ ]:
ds.layers[0].map(geometry=g)

Finally, you can call the intersect function on the dataset object to see a dictionary of values.

In [ ]:
ds.intersect(geometry=g)

### Layer Intersections

Intersections against layers work in the same manner. Here we show an intersect between an area along Brazil's coast and a Mangrove biomass density Layer.

In [ ]:
atts={'geojson': {'type': 'FeatureCollection',
  'features': [{'type': 'Feature',
    'properties': {},
    'geometry': {'type': 'Polygon',
     'coordinates': 

[
          [
            [
              -52.16308593749999,
              -1.669685500986571
            ],
            [
              -46.9775390625,
              -1.669685500986571
            ],
            [
              -46.9775390625,
              0.7909904981540058
            ],
            [
              -52.16308593749999,
              0.7909904981540058
            ],
            [
              -52.16308593749999,
              -1.669685500986571
            ]
          ]
        ]
                 }}]}}

g = Skydipper.Geometry(attributes=atts)


In [ ]:
ly = Skydipper.Layer('3ef039ba-29c7-459d-aadd-66d6e6cc8f5e')
ly

In [ ]:
ly.map(geometry=g)

In [ ]:
ly.intersect(geometry=g)

## ImageCollection: Search for satellite images

You can search for Landsat-8 and Sentinel-2 imagery using the ImageCollection module as follows:

In [ ]:
ic = Skydipper.ImageCollection(lon=28.3, lat=-16.6, start='2018-08-01', end='2018-08-10') # n.b. lon/lat are temporarily flipped

In [ ]:
ic

In [ ]:
type(ic)

All types of pythonic list mainipulation are supported on the image collection results. Including iteration:

In [ ]:
for i in ic[0:2]:
    print(i)

...and subsetting to access individual Images.

## Images: Individual satellite tiles

In [ ]:
i = ic[0]
i

In [ ]:
type(i)

You can access the Image attributes, which indicate the provinence of the tile. 

In [ ]:
i.attributes

### Mapping Satellite Images

You can also display web-map tiles and the bounding-box of the satellite image.

In [ ]:
i.map()

### Classification of individual Satellite Images

In [ ]:
classified = i.classify()
classified

In [ ]:
classified.map()

### Composite and classify satellite image collections

First grab a collection, specifying a point and time period.

In [ ]:
ic = Skydipper.ImageCollection(lon=28.271979, lat=-16.457814, start='2018-06-01', end='2018-06-20')
ic

Next ask for a composite image based on your collection. By default it will be for Sentinel-2, but you may change this via an argument to Landsat.

In [ ]:
i = ic.composite()
i

You can then map your composite image.

In [ ]:
i.map()

You can take your composite image, and apply a land cover classifier based on our pre-trained models. These are Deepvel, and Segnet.

In [ ]:
c = i.classify(model_type='segnet', version='v2')
c

In [ ]:
c.map()

## Construct Widgets

A demo of how to use Skydipper to construct widgets like those on our projects such as [Global Forest Watch Dashboard pages](https://www.globalforestwatch.org/dashboards/country/BRA).

### Tree cover example

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Get a datatable (Hansen)
table = Skydipper.Table('a20e9c0e-8d7d-422f-90f5-3b9bca355aaf')
table

In [ ]:
iso = 'BRA'
administration = 1

sql = f"""
        SELECT 
            SUM(area_extent) as value,
            SUM(area_admin) as total_area 
        FROM data 
        WHERE iso = '{iso}'
        AND adm1 = {administration}
        AND thresh = 30 
        AND polyname = 'admin'
"""

results = table.query(sql=sql)
results

In [ ]:
sizes = [results.value[0], results.total_area[0] - results.value[0]]
colors = ['green','grey']
labels = ['Tree cover', 'Non-forest']

fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%',
        shadow=False, startangle=90, colors=colors)
ax1.axis('equal')
centre_circle = plt.Circle((0,0),0.75,color='black', fc='white',linewidth=0.5)
fig1 = plt.gcf()
fig1.gca().add_artist(centre_circle)
plt.suptitle('Tree cover extent')
plt.title(f'in {iso}/{administration}')
plt.show()

### Tree cover loss example

In [ ]:
sql = """
SELECT 
    polyname, year_data.year as year, 
    SUM(year_data.area_loss) as area

FROM data 
WHERE polyname = 'admin'
AND thresh= 30
GROUP BY polyname, iso, nested(year_data.year)
"""
global_loss = table.query(sql=sql)
global_loss.head()

In [ ]:
iso='BRA'
loss_data = list(global_loss[global_loss['iso'] == f'{iso}']['area'])
years = list(global_loss[global_loss['iso'] == f'{iso}']['year'])

width = 0.66
fig, ax = plt.subplots()
rects1 = ax.bar(years, loss_data, width, color='#FE5A8D')

# add some text for labels, title and axes ticks
ax.set_ylabel('Loss extent (ha)')
ax.set_title(f'Loss by year in {iso}')
plt.show()

## Creating a local backup of Data objects

Save a local backup of a collection to a specified path. This creates a folder containing a JSON for each dataset and it's associated Layers, Metadata and Vocabularies.

In [ ]:
col = Skydipper.Collection(app=['gfw'], env='production')

In [ ]:
path = './LMI-BACKUP'

In [ ]:
col.save(path)

## Load Data objects from local backup

You can also load a previous version from local backup.

In [ ]:
import os

In [ ]:
files = os.listdir(path)[0:3]
files

In [ ]:
ds_id = files[0].split('.json')[0]
ds_id

In [ ]:
dataset = Skydipper.Dataset(ds_id)

In [ ]:
dataset

In [ ]:
backup = dataset.load(path)

In [ ]:
backup